# Quality Detector

In this script I will create toy CNN that is supposed to detect whether the input image is of a good quality or has artefacts.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Data Preprocessing

I will start by loading two of the images in. Then I will select from the originals each only one. Aftwards, I will select the 500 images in good and bad quality from the two image and create the classification label for it.

In [2]:
P3_X = np.load('/home/cloud/MRT_Data/unziped/out/P3_X.npy')
P3_Y = np.load('/home/cloud/MRT_Data/unziped/out/P3_Y.npy')
P14_X = np.load('/home/cloud/MRT_Data/unziped/out/P14_X.npy')
P14_Y = np.load('/home/cloud/MRT_Data/unziped/out/P14_Y.npy')

#### Select images from Y only once
To match the number of distorted images in X, the images in Y are each repeated five times subsequently. To get unique images I will only select the first once.

In [3]:
P3_Y = P3_Y[[i for i in range(0,P3_Y.shape[0],5)],:,:,:]

In [4]:
P14_Y = P14_Y[[i for i in range(0,P14_Y.shape[0],5)],:,:,:]

For the distorted pictures saved in P3_X and P14_X I will select on of the five distorted images randomly.

In [5]:
P3_X = P3_X[[i + np.random.randint(0,4) for i in range(0,P3_X.shape[0],5)], :, :,:]

In [6]:
P14_X = P14_X[[i + np.random.randint(0,4) for i in range(0,P14_X.shape[0],5)], :, :,:]

Combine all images and create class vector

In [7]:
shape_x = (4 * P3_X.shape[0],) + P3_X.shape[1:4]

In [8]:
X = np.empty(shape_x,dtype='uint8')
X[0:811,:,:,:] = P3_Y
X[811:1622,:,:,:] = P14_Y
X[1622:2433,:,:,:] = P3_X
X[2433:3244,:,:,:] = P14_X

In [9]:
Y = np.array([1] * 1622 + [0] * 1622 , dtype='uint8')

Shuffle both arrays randomly!

In [10]:
shuffle_index = np.random.choice(range(0,Y.shape[0]), size = Y.shape[0], replace = False)

In [11]:
X = X[shuffle_index,:,:,:]

In [12]:
Y = Y[shuffle_index,]

# Create train and test and validation set

In [13]:
X_train = X[1:2270, :,:,:]
X_eval = X[2270:3244,:,:,:]

Y_train = Y[1:2270,]
Y_eval = Y[2270:3244,]

# Create and train CNN

Create the CNN model

In [14]:
def cnn_model_fn(features, labels, mode):
    print(features['x'].shape)
    print(labels.shape)
   

Load train and test data

In [15]:
train_data = X_train
train_labels = Y_train
eval_data = X_eval
eval_labels = Y_eval

Create the estimator

In [16]:
noise_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/noise_classifier")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/noise_classifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f24ab670780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Set Up a Logging Hook

In [17]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

Train the model

In [18]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [19]:
noise_classifier.train(
    input_fn=train_input_fn,
    steps=100)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
(100, 256, 256, 3)
(100,)
INFO:tensorflow:Done calling model_fn.


ValueError: model_fn should return an EstimatorSpec.